In [0]:
from pyspark.sql import functions as F

filePath = "dbfs:/FileStore/tables/Files/names.csv"
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(filePath)

nulls_df = df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(f"{c}_nulls") for c in df.columns
])

filled_df = df.fillna({
    "name": "Unknown",
    "birth_name": "Unknown",
    "height": 0,
    "bio": "Brak danych",
    "spouses": 0,
    "divorces": 0,
    "spouses_with_children": 0,
    "children": 0
})

dropped_df = filled_df.drop("death_details")

regex_replace_df = df.withColumn("clean_name", F.regexp_replace(F.col("name"), "\\d", ""))

regex_extract_df = df.withColumn("birth_year", F.regexp_extract(F.col("date_of_birth"), r"^(\d{4})", 1))

ifnull_df = df.withColumn("safe_height", F.expr("ifnull(height, 170)"))
nullif_df = df.withColumn("name_diff", F.expr("nullif(name, birth_name)"))
replaced_df = df.withColumn("updated_bio", F.expr("replace(bio, 'actor', 'performer')"))

avg_children = df.select(F.avg("children").alias("avg_children"))
max_divorces = df.select(F.max("divorces").alias("max_divorces"))
unique_birth_places = df.select(F.countDistinct("place_of_birth").alias("unique_birth_places"))

places_df = df.withColumn("birth_places_array", F.split(F.col("place_of_birth"), ", "))
exploded_places_df = places_df.withColumn("exploded_place", F.explode(F.col("birth_places_array")))

contains_df = places_df.withColumn(
    "was_born_in_houston", 
    F.array_contains(F.col("birth_places_array"), "Omaha")
)

print("NULLS:")
nulls_df.show()

print("CLEANED NAME:")
regex_replace_df.select("name", "clean_name").show(5)

print("EXTRACTED BIRTH YEAR:")
regex_extract_df.select("date_of_birth", "birth_year").show(5)

print("AGGREGATES:")
avg_children.show()
max_divorces.show()
unique_birth_places.show()

print("EXPLODED PLACES:")
exploded_places_df.select("name", "exploded_place").show(truncate=False)

print("BORN IN OMAHA?")
contains_df.select("name", "birth_places_array", "was_born_in_houston").show()

NULLS:
+------------------+----------+----------------+------------+---------+-------------------+-------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+-------------+--------------+---------------------------+--------------+
|imdb_name_id_nulls|name_nulls|birth_name_nulls|height_nulls|bio_nulls|birth_details_nulls|date_of_birth_nulls|place_of_birth_nulls|death_details_nulls|date_of_death_nulls|place_of_death_nulls|reason_of_death_nulls|spouses_string_nulls|spouses_nulls|divorces_nulls|spouses_with_children_nulls|children_nulls|
+------------------+----------+----------------+------------+---------+-------------------+-------------------+--------------------+-------------------+-------------------+--------------------+---------------------+--------------------+-------------+--------------+---------------------------+--------------+
|                 0|         0|               0|       49958|     

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf

# Standardowa funkcja UDF (korzystająca z Pythona)
def cm_to_inches(height_cm):
    if height_cm is None:
        return 0.0
    return round(height_cm * 0.393701, 2)

# Rejestracja funkcji jako UDF
cm_to_inches_udf = udf(cm_to_inches, DoubleType())

# Użycie UDF w DataFrame
df_with_inches = df.withColumn("height_inches", cm_to_inches_udf(F.col("height")))
df_with_inches.select("height", "height_inches").show()

+------+-------------+
|height|height_inches|
+------+-------------+
|   177|        69.69|
|   174|         68.5|
|   166|        65.35|
|   170|        66.93|
|   179|        70.47|
|   178|        70.08|
|   173|        68.11|
|   178|        70.08|
|   165|        64.96|
|   191|         75.2|
|   160|        62.99|
|   170|        66.93|
|   163|        64.17|
|   173|        68.11|
|  null|          0.0|
|   164|        64.57|
|   175|         68.9|
|   182|        71.65|
|   187|        73.62|
|   161|        63.39|
+------+-------------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
import pandas as pd

# Pandas UDF do wyciągania nazwiska z pełnego imienia
@pandas_udf(StringType())
def extract_last_name(birth_name: pd.Series) -> pd.Series:
    return birth_name.fillna("").apply(lambda x: x.strip().split(" ")[-1] if x else "Unknown")

# Użycie
df_with_last_name = df.withColumn("last_name", extract_last_name(F.col("birth_name")))
df_with_last_name.select("birth_name", "last_name").show(truncate=False)


+-----------------------------------+------------+
|birth_name                         |last_name   |
+-----------------------------------+------------+
|Frederic Austerlitz Jr.            |Jr.         |
|Betty Joan Perske                  |Perske      |
|Brigitte Bardot                    |Bardot      |
|John Adam Belushi                  |Belushi     |
|Ernst Ingmar Bergman               |Bergman     |
|Ingrid Bergman                     |Bergman     |
|Humphrey DeForest Bogart           |Bogart      |
|Richard Walter Jenkins             |Jenkins     |
|James Francis Cagney               |Cagney      |
|Frank James Cooper                 |Cooper      |
|Ruth Elizabeth Davis               |Davis       |
|Doris Mary Ann Kappelhoff          |Kappelhoff  |
|Olivia Mary de Havilland           |Havilland   |
|James Byron Dean                   |Dean        |
|Georges Henri Jean-Baptiste Delerue|Delerue     |
|Marie Magdalene Dietrich           |Dietrich    |
|Issur Herschelevitch Danielovi